# Декораторы

### Замыкания (closures)



Замыкание (*closures*) — функция с расширенной областью видимости, которая охватывает не глобальные
переменные, на которые есть ссылки в теле функции, хотя в теле функции они (переменные) не определенны.
Еще раз обратите внимание на то, что функция должна иметь доступ к неглобальным переменным. Т.е. к
переменным объявленным в другой (объемлющей) функции.

In [ ]:
def add(q, w):
    return q + w

a = add
print(a(2, 3))

In [ ]:
"""Область видимости параметра n принадлежит внешней функции, однако
вложенная функция может его использовать."""

def calculate_pow(n): # Объемлющая функция
    def calculate(number):
        print(locals())
        # Вложенная функция, которая использует переменную объемлющей
        return number ** n
    return calculate # Возврат вложенной функции

f = calculate_pow(3) # Вызов объемлющей функции




In [ ]:
f

In [ ]:
number_one = f(2) # Вызов вложенной функции
number_two = f(5)
print(number_one)
print(number_two)

In [ ]:
func = calculate_pow(5)

In [ ]:
func(3)

In [ ]:
f(4)

внутренняя функция и является **замыканием**. В Python область видимости объемлющей функции
сохраняется для внутренней функции. Но не всех, а только для тех переменных которые используются во вложенной
функции. Такие переменные объемлющей функции называются свободными переменными.

In [ ]:
# проблемы при построении функции замыкания
def fibonacci():
    first_number = 0
    second_number = 1
    def get_next():
        next_number = second_number + first_number # UnboundLocalError
        first_number = second_number
        second_number = next_number
        return next_number
    return get_next

In [ ]:
f = fibonacci()

for i in range(10):
    print(f(), end = " ")

### Использование модификатора nonlocal

In [ ]:
def fibonacci():
    first_number = 0
    second_number = 1
    def get_next():
        nonlocal second_number # указание, что переменные нелокальны
        nonlocal first_number
        next_number = second_number + first_number
        first_number = second_number
        second_number = next_number
        return next_number
    return get_next


In [ ]:
f = fibonacci()

for i in range(10):
    print(f(), end = " ")


## Декораторы функций

### Декоратор — это функция, которая позволяет обернуть другую функцию для расширения её функциональности без непосредственного изменения её кода.

**Декоратор** — вызываемый объект, который принимает другую функцию в качестве аргумента. Декоратор может производить
какие-то операции с функцией и возвращает либо ее саму, либо
другую, заменяющую ее функцию или вызываемый объект.
Декораторы функций связывают имя функции с другим вызываемым объектом на этапе определения функции, добавляя
дополнительный уровень логики, которая управляет функциями и методами или выполняет некоторые действия в случае их вызова.

Декораторы функций могут быть использованы для:

* Перехват вызова функций и выполняющие необходимые операции с функцией. Например, регистрация в прикладном компоненте, хронометраж.
* Декоратор может полностью заменить объект функции на другой. Или модифицировать объект функции.

Т.е. с помощью декоратора можно дополнять декорируемую функцию, можно использовать результаты ее работы, а можно полностью ее
заменить.

**Что может выступать в качестве декоратора в Python?**

Декоратор сам по себе является вызываемым объектом, который возвращает вызываемый объект. Т.е. в качестве
декоратора может использоваться любой объект, который реализует протокол вызова.
Если рассмотреть декораторы с технической точки зрения, то декораторы в Python - это синтаксическое упрощение
(синтаксический сахар) при описания объекта, который может управлять объектом функцией.
Однако, из-за своей наглядности, они применяются очень часто.

In [ ]:
my_function = []

def add_function(func):
    """функция принимает на вход любой объект, добавляет
        его в список my_function и возвращает этот объект.
    """
    my_function.append(func)
    return func

@add_function # Применение созданной функции в качестве декоратора
def summ(x, y): # Декорируемая функция summ = add_function(sum)
    return x + y
# summ = add_function(summ)

@add_function
def mul(x, y): # Декорируемая функция
    return x * y
# mul = add_function(mul)

print(my_function)

In [ ]:
mul(4, 5)

In [ ]:
def div(q, w):
    return q / w


In [ ]:
div(5, 6)


In [ ]:
div = add_function(div)
# @add_function
# def div(q, w):
#     return q / w


In [ ]:
print(my_function)

декораторы - не более чем упрощение синтаксиса. Однако, из-за наглядности и удобства использования, их применяют
довольно часто.

**Внимание!** Декораторы выполняются на этапе создания функции. Декораторы не вызываются при вызове декорируемой
функции


### Передача параметров для декорируемой функции.
Пример декоратора, который использует результат вызова декорируемой функции

In [ ]:
def to_str(func):
    def get_str(*args, **kwargs): # Функция, которая принимает аргументы для декор.функции
        return str(func(*args, **kwargs))
    return get_str # Теперь возвращается другая функция

@to_str
def suma(x, y):
    return x + y
print("Summa = " + suma(3, 4))

# suma = to_str(suma)
# suma = get_str(*args, **kwargs) -> str(func(*args, **kwargs))

In [ ]:
'h' + 7

в результате вызова функции декоратора возвращается уже не декорируемая функция, а иной вызываемый объект (функция get_str)

In [ ]:
print(suma)

In [ ]:
@to_str
def div(q, w):
    return q / w

In [ ]:
div

In [ ]:
div(3, 5)

### Как это можно исправить?

In [ ]:
def trace(func):
    """Декоратор trace выводит на экран сообщение с
        информацией о вызове декорируемой функции."""
    def inner(*args, **kwargs):
        """Inner doc"""
        print(f'name: {func.__name__}, args: {args}, kwargs: {kwargs}')
        return func(*args, **kwargs)
    return inner

In [ ]:
@trace # identity = trace(identity)
def identity(x):
    """I do nothing useful."""
    return x

identity(50)



In [ ]:
identity

In [ ]:
help(identity) # Help on function inner

In [ ]:
def identity(x):
    """I do nothing useful."""
    return x

identity.__name__, identity.__doc__

In [ ]:
# @trace 
# def identity(x):
identity = trace(identity)


In [ ]:
identity.__name__, identity.__doc__

In [ ]:
# У любой функции в Python есть атрибут __module__ ,
# содержащий имя модуля, в котором функция была определена.
identity.__module__

In [ ]:
import math
print(math.cos.__module__)

In [36]:
# установим “правильные” значения в атрибуты декорируемой функции:
def trace(func):
    """Декоратор trace выводит на экран сообщение с
        информацией о вызове декорируемой функции."""
    def inner(*args, **kwargs):
        """Inner doc"""
        print(f'name: {func.__name__}, args: {args}, kwargs: {kwargs}')
        return func(*args, **kwargs)
    inner.__module__ = func.__module__
    inner.__name__ = func.__name__
    inner.__doc__ = func.__doc__
    return inner


@trace
def identity(x):
    """I do nothing useful."""
    return x

@trace
def hello(x):
    print(f"hello {x}")

print('identity', identity)
print('hello', hello)

hello("you")

identity <function trace.<locals>.inner at 0x1064b3420>
hello <function trace.<locals>.inner at 0x1064b3380>
name: hello, args: ('you',), kwargs: {}
name: hello, args: ('you',), kwargs: {}
hello you


In [ ]:
# Проверим
identity.__name__, identity.__doc__

In [ ]:
dir(identity)

In [ ]:
identity.__closure__

В модуле **functools** из стандартной библиотеки Python есть функция, реализующая логику копирования
внутренних атрибутов


In [37]:
import functools

def trace(func):
    """Декоратор trace выводит на экран сообщение с
        информацией о вызове декорируемой функции."""
    def inner(*args, **kwargs):
        """Inner doc"""
        print(f'name: {func.__name__}, args: {args}, kwargs: {kwargs}')
        return func(*args, **kwargs)
    functools.update_wrapper(inner, func)
    return inner

@trace
def identity(x):
    """I do nothing useful."""
    return x

print(identity)
identity.__name__, identity.__doc__

<function identity at 0x1064b2840>


('identity', 'I do nothing useful.')

In [38]:
# То же самое можно сделать с помощью декоратора wraps

def trace(func):
    """Декоратор trace выводит на экран сообщение с
        информацией о вызове декорируемой функции."""
    @functools.wraps(func)
    def inner(*args, **kwargs):
        """Inner doc"""
        print(f'name: {func.__name__}, args: {args}, kwargs: {kwargs}')
        return func(*args, **kwargs)
    return inner

@trace
def identity(x):
    """I do nothing useful."""
    return x

print(identity.__name__, identity.__doc__)
print(identity(34))

identity I do nothing useful.
name: identity, args: (34,), kwargs: {}
34


### К одной функции можно применить множество декораторов

In [40]:
def bread(func):
    def wrapper():
        print()
        func()
        print("<\\______/>")
    return wrapper

def ingredients(func):
    def wrapper():
        print("#помидоры#")
        func()
        print("~салат~")
    return wrapper

def sandwich(food="--ветчина--"):
    print(food)

sandwich()

--ветчина--


In [41]:
sandwich = bread(ingredients(sandwich))
sandwich()


#помидоры#
--ветчина--
~салат~
<\______/>


In [42]:
@bread
@ingredients
def sandwich(food="--ветчина--"):
    print(food)

sandwich()


#помидоры#
--ветчина--
~салат~
<\______/>


In [43]:


@ingredients
@bread
def sandwich(food="--ветчина--"):
    print(food)

sandwich()

#помидоры#

--ветчина--
<\______/>
~салат~


In [54]:
def extra_food(func):
    def wrapper(*args, **kwargs):
        func()
        func()
    return wrapper

# важен порядок декорирования
@bread
@ingredients
@ingredients
def sandwich(food="--ветчина--"):
    print(food)

sandwich()





#помидоры#
#помидоры#
--ветчина--
~салат~
~салат~
<\______/>


### Аргументы декораторов
Обе разновидности декораторов (декораторы на основе функций и декораторы на
основе классов) могут принимать дополнительные аргументы.
Данный механизм реализован следующим образом: аргументы переданные
декоратору, на самом деле, передаются объекту, который вернет декоратор. А уже
возвращенный декоратор будет применен к декорируемому объекту.
Т.е. если использовать декоратор с параметрами, то, в качестве декоратора, стоит
использовать вызываемый объект, который вернет декоратор.

```
@my_decorator
def func(*args):
    ...
```
=>

`func = my_decorator(func)`

Для декоратора с параметрами, сохраняется логика, но добавляется
промежуточный уровень обработки
```
@my_decorator(x, y)
def func(*args):
    ...
```
=>
```
deco = my_decorator(x, y)
func = deco(func)
```

In [55]:
@bread('Hi')
@ingredients
def sandwich(food="--ветчина--"):
    print(food)

sandwich()

TypeError: bread.<locals>.wrapper() takes 0 positional arguments but 1 was given

In [56]:
def decorator_with_arguments(deco_arg1, deco_arg2):
    print("Я создаю декораторы! аргументы:", deco_arg1, deco_arg2)
    def my_decorator(func):
        print("Я и есть декоратор. Аргументы извне:", deco_arg1, deco_arg2)
        # Не перепутайте аргументы декораторов с аргументами функций!
        def wrapped(func_arg1, func_arg2):
            print ("Я - обёртка вокруг декорируемой функции.\n"
            "И я имею доступ ко всем аргументам\n"
            "\t- и декоратора: {0} {1}\n"
            "\t- и функции: {2} {3}\n"
            "Теперь я могу передать нужные аргументы дальше"
            .format(deco_arg1, deco_arg2, func_arg1, func_arg2))
            return func(func_arg1, func_arg2)
        return wrapped
    return my_decorator


In [57]:
@decorator_with_arguments("Леонард", "Шелдон")
def decorated_function_with_arguments(_arg1, _arg2):
    print ("Я - декорируемая функция и я знаю только о своих аргументах: {0}"
    " {1}".format(_arg1, _arg2))

print('-------------------- start ---------------')
decorated_function_with_arguments("Раджеш", "Говард")

Я создаю декораторы! аргументы: Леонард Шелдон
Я и есть декоратор. Аргументы извне: Леонард Шелдон
-------------------- start ---------------
Я - обёртка вокруг декорируемой функции.
И я имею доступ ко всем аргументам
	- и декоратора: Леонард Шелдон
	- и функции: Раджеш Говард
Теперь я могу передать нужные аргументы дальше
Я - декорируемая функция и я знаю только о своих аргументах: Раджеш Говард


In [60]:
# Декорирование добавит функцию в словарь workers.
# Ключом будет выступать строка (параметр декоратора),
# а значением - декорируемая функция.

workers = {}

def link(address=None):
    def add_worker(func):
        workers[address] = func
        def get_answer(*args, **kwargs):
            return str(func(*args, **kwargs))
        return get_answer
    return add_worker

In [61]:
@link("/main")
def main_page():
    return "Hello word page"

@link("/main/goods")
def get_goods(list_goods):
    return list_goods

print(workers)

{'/main': <function main_page at 0x1064b1b20>, '/main/goods': <function get_goods at 0x1064b3240>}


In [62]:
@link()
def world():
    return "Hello world"
print(workers)

{'/main': <function main_page at 0x1064b1b20>, '/main/goods': <function get_goods at 0x1064b3240>, None: <function world at 0x1064b2d40>}


### Некоторые декораторы стандартной библиотеки
Рассмотрим некоторые декораторы в стандартной библиотеке:

**functools.lru_cache** — Встроенная реализация  мемоизации для пользовательских рекурсивных функций

**functools.singledispatch** — Реализация обобщенных (перегруженных) функций в Python

### Использование декоратора functools.lru_cache
Данный декоратор используется для реализации приема мемоизации («memoization»). Его смысл заключается в сохранении
параметра метода и его возвращаемого значения в быстром хранилище (словарь). Этот прием позволит значительно ускорить
вычисление некоторых рекурсивных функций. Так как, если в этом словаре уже будут параметры, с которыми вызывалась функция, то
она не будет вычисляться, а ответ возьмется из словаря.
У данного декоратора существует два параметра:

**maxsize** — сколько результатов вызова хранить. Для достижения максимальной производительности рекомендуется использовать два
в целой степени. По умолчанию maxsize=128

**typed** — по-разному хранить параметры разных типов. Т.е.
integer и float хранятся по разному. Например для чисел 3 и 3.0 данные будут храниться, как разные.

**Внимание!** Параметры функции должны быть хешируемого типа.

In [63]:
import time
def fibonacci(n):
    if n == 0:
        return 0
    if n == 1:
        return 1
    return fibonacci(n-1) + fibonacci(n-2)

start = time.time()
res = fibonacci(36) # 12.167090892791748  75 будет кардинально дольше! 2 в степени n!
print(time.time() - start)

1.6370759010314941


In [64]:
import functools
import time

@functools.lru_cache()
def fibonacci(n):
    if n == 0:
        return 0
    if n == 1:
        return 1
    return fibonacci(n-1) + fibonacci(n-2)

start = time.time()
res = fibonacci(250)
print(res)
print(time.time() - start)

7896325826131730509282738943634332893686268675876375
0.0001690387725830078


In [65]:
def fibonacci(n):
    if n == 0:
        return 0
    if n == 1:
        return 1
    print(n)
    return fibonacci(n-1) + fibonacci(n-2)


res = fibonacci(10)

10
9
8
7
6
5
4
3
2
2
3
2
4
3
2
2
5
4
3
2
2
3
2
6
5
4
3
2
2
3
2
4
3
2
2
7
6
5
4
3
2
2
3
2
4
3
2
2
5
4
3
2
2
3
2
8
7
6
5
4
3
2
2
3
2
4
3
2
2
5
4
3
2
2
3
2
6
5
4
3
2
2
3
2
4
3
2
2


### Свой декаратор для кеширования

In [ ]:
def cache(func):
    data = {}
    def wrapper(*args):
        if args in data:
            return data[args]
        else:
            result = func(*args)
            data[args] = result
            return result
    return wrapper


In [ ]:
@cache
def fibonacci(n):
    if n in (0, 1):
        return n
    else:
        return fibonacci(n-1) + fibonacci(n-2)

start = time.time()
res = fibonacci(250)
print(res)
print(time.time() - start)

### Использование декоратора functoois.singledispatch
Данный декоратор используется для реализации приема
перегрузки методов. В Python не реализован механизм перегрузки
методов в зависимости от типа параметра. Однако необходимость в
подобном механизме возникает. В этом случае можно использовать
встроенный декоратор **functoois.singledispatch**. Функция, к которой
применен данный декоратор, становится обобщенной.
После этого вы можете зарегистрировать несколько других
функций для обработки аргумента своего типа. Для этого другие
функции следует пометить декоратором вида:

**decorate_function_name.register (date_type)**

**decorate_function_name** — имя обобщенной функции

**date_type** — тип данных

**Внимание!** Использовать декоратор можно только для работы с первым аргументом.

In [ ]:
import functools
import numbers

@functools.singledispatch
def add(a, b): # функция add будет обобщенной
    return a + b

@add.register(numbers.Integral) # если первый параметр add - целое число
def integer_add(a, b):
    print("Call for integer")
    return a + b

@add.register(numbers.Real) # если первый параметр add - вещественное число
def float_add(a, b):
    print("Call for float")
    return a + b

In [ ]:
# вызываются разные функции по одному и тому же имени
print(add(3, 5))
print(add(1.5, 0.5))

In [ ]:
# Использовать декоратор можно только для работы с первым аргументом.
print(add(3.0, 5))
print(add(15, 0.5))


In [ ]:
functools.singledispatch.__doc__